# 04b - Regression Models for Risk Score Prediction

**Objetivo**: Desarrollar modelos de regresión para predecir el `composite_risk_score` continuo
 
**Modelos a desarrollar**:
- Linear Regression (baseline)
- Random Forest Regressor
- XGBoost Regressor
- Support Vector Regressor
- Neural Network Regressor

---

## Importar Librerías

In [17]:
import sys
import os
sys.path.append('../src/modeling')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import gc
warnings.filterwarnings('ignore')
from sklearn.impute import SimpleImputer

# Importar pipeline de regresión
from regression_pipeline import RegressionPipeline
from model_utils import load_processed_data, save_model_artifacts
import mlflow
import mlflow.sklearn

## Configuraciones

In [2]:
# Configuración
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [3]:
print("✅ Librerías y módulos importados correctamente")
print(f"📅 Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Librerías y módulos importados correctamente
📅 Fecha de ejecución: 2025-06-15 09:52:31


In [3]:
# Configuración MLflow
mlflow.set_experiment("alzheimer_multimodal_monitoring")

print("🔧 MLflow configurado para modelos de regresión")
print("🎯 Target: composite_risk_score (regresión continua)")


🔧 MLflow configurado para modelos de regresión
🎯 Target: composite_risk_score (regresión continua)


In [4]:
# Agrega después de tu configuración:
experiment = mlflow.get_experiment_by_name("alzheimer_multimodal_monitoring")
print(f"ID del experimento: {experiment.experiment_id}")
print(f"Ubicación: {experiment.artifact_location}")

ID del experimento: 503314857977141831
Ubicación: file:///E:/usuarios/alumno/Escritorio/Alzheimer-Multimodal-Monitoring/notebooks/mlruns/503314857977141831


## Cargar datos procesados

In [5]:
# Cargar datos procesados
try:
    df = pd.read_csv('../data/processed/features/alzheimer_features_selected_20250605.csv')
    print(f"📊 Dataset cargado: {df.shape}")
    print(f"📈 Registros con score válido: {df['composite_risk_score'].notna().sum()}")
    
    # Información del target
    target_stats = df['composite_risk_score'].describe()
    print(f"\n📊 ESTADÍSTICAS DEL TARGET:")
    print(f"   • Media: {target_stats['mean']:.3f}")
    print(f"   • Desviación estándar: {target_stats['std']:.3f}")
    print(f"   • Rango: [{target_stats['min']:.3f}, {target_stats['max']:.3f}]")
    
except FileNotFoundError:
    print("❌ Error: Archivo no encontrado")
    print("💡 Ejecuta primero el notebook de feature engineering")

📊 Dataset cargado: (48466, 189)
📈 Registros con score válido: 48466

📊 ESTADÍSTICAS DEL TARGET:
   • Media: 0.367
   • Desviación estándar: 0.213
   • Rango: [0.000, 0.929]


In [6]:
col_nonnum = df.select_dtypes(exclude=['number'])
cant_nonnum = col_nonnum.shape[1]
print(f"cantidad de columnas no numericas: {cant_nonnum}")
print(list(col_nonnum))

cantidad de columnas no numericas: 9
['gender', 'update_stamp_apoe_normalized', 'update_stamp_apoe', 'PTGENDER', 'subject_id_activity', 'PTID_apoe', 'EDUCATION_LEVEL', 'gender_normalized', 'risk_category']


In [7]:
# Inicializar pipeline de regresión
regression_pipeline = RegressionPipeline(
    target_col='composite_risk_score',
    test_size=0.2,
    random_state=42
)

print("🔧 Pipeline de regresión inicializado")
print(f"   • Target: {regression_pipeline.target_col}")
print(f"   • Test size: {regression_pipeline.test_size}")

🔧 Pipeline de regresión inicializado
   • Target: composite_risk_score
   • Test size: 0.2


## Separación de datos en características (X) y variable objetivo (y)

In [8]:
# Separar características (X) y variable objetivo (y)
X = df.drop(columns=['composite_risk_score'])  # Todas las columnas excepto el target
y = df['composite_risk_score']  # Solo la columna objetivo

# Verificar las dimensiones
print(f"🔍 Dimensiones de los datos:")
print(f"   • X: {X.shape} (features)")
print(f"   • y: {y.shape} (target)")

🔍 Dimensiones de los datos:
   • X: (48466, 188) (features)
   • y: (48466,) (target)


In [9]:
print("Valores faltantes en X:", X.isna().sum().sum())
print("Valores faltantes en y:", y.isna().sum())

Valores faltantes en X: 1077109
Valores faltantes en y: 0


## Ejecución del Pipeline y Manejo de NaN

In [16]:
from joblib import Memory
import mlflow

# Limpiar caché de joblib
mem = Memory(location=None)
mem.clear()

# Limpiar caché de MLflow (opcional)
mlflow.end_run()

[Memory(location=None)]: Flushing completely the cache


In [19]:
# 1. Eliminar columnas no numéricas
non_numeric = ['gender', 'update_stamp_apoe_normalized', 'update_stamp_apoe', 
              'PTGENDER', 'subject_id_activity', 'PTID_apoe', 'EDUCATION_LEVEL', 
              'gender_normalized', 'risk_category']
X = X.drop(columns=non_numeric, errors='ignore')  # 'errors=ignore' evita errores si alguna columna no existe

# 2. Reemplazar infinitos por NaN
print(f"Valores infinitos antes: {np.isinf(X.values).sum()}")
X.replace([np.inf, -np.inf], np.nan, inplace=True)

# 3. Identificar y eliminar columnas completamente vacías
empty_cols = X.columns[X.isna().all()].tolist()
print(f"\n🔥 Columnas completamente vacías ({len(empty_cols)}): {empty_cols}")
X = X.drop(columns=empty_cols)

# 4. Imputar valores faltantes
print(f"\nValores NaN en X antes de imputar: {X.isna().sum().sum()}")
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# 5. Crear nuevo DataFrame manteniendo solo las columnas no vacías
X = pd.DataFrame(X_imputed, columns=X.columns)

# 6. Verificar resultados
print(f"\nValores infinitos después: {np.isinf(X.values).sum()}")
print(f"Valores NaN después de imputar: {X.isna().sum().sum()}")
print(f"Nuevas dimensiones: {X.shape}")

# Liberar memoria explícitamente
gc.collect()

# Ejecutar el pipeline completo de regresión para todos los modelos
results = regression_pipeline.run_regression_pipeline(
    X,  # Features 
    y,  # Target 
    optimize_hyperparams=False,  # No optimizar hiperparámetros
    cross_validate=True          # Realizar validación cruzada
)

print("✅ Pipeline completo ejecutado!")

Valores infinitos antes: 0

🔥 Columnas completamente vacías (0): []

Valores NaN en X antes de imputar: 0

Valores infinitos después: 0
Valores NaN después de imputar: 0
Nuevas dimensiones: (48466, 169)

🚀 INICIANDO PIPELINE DE REGRESIÓN
   • Dimensiones entrenamiento: (38772, 169)
   • Dimensiones test: (9694, 169)

 Entrenando linear_regression...
📊 RMSE: 0.0034
📊 MAE: 0.0014
📊 R²: 0.9997
📊 CV RMSE: 0.0034 ± 0.0001


MlflowException: Failed to convert metric value to float: can only convert an array of size 1 to a Python scalar

## Comparación de modelos

In [ ]:
# Comparación de modelos
models_comparison = {
    'Linear Regression': lr_results,
    'Random Forest': rf_results,
    'XGBoost': xgb_results,
    'SVR': svr_results,
    'Neural Network': nn_results
}

# Crear DataFrame de comparación
comparison_df = pd.DataFrame({
    model_name: {
        'R² Score': results['r2_score'],
        'RMSE': results['rmse'],
        'MAE': results['mae'],
        'Training Time (s)': results.get('training_time', 0)
    }
    for model_name, results in models_comparison.items()
}).T

print("\n📊 COMPARACIÓN DE MODELOS DE REGRESIÓN")
print("=" * 60)
print(comparison_df.round(4))


In [ ]:
# Visualizaciones de comparación
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# R² Score comparison
comparison_df['R² Score'].plot(kind='bar', ax=axes[0,0], color='skyblue')
axes[0,0].set_title('R² Score por Modelo')
axes[0,0].set_ylabel('R² Score')
axes[0,0].tick_params(axis='x', rotation=45)

# RMSE comparison
comparison_df['RMSE'].plot(kind='bar', ax=axes[0,1], color='lightcoral')
axes[0,1].set_title('RMSE por Modelo')
axes[0,1].set_ylabel('RMSE')
axes[0,1].tick_params(axis='x', rotation=45)

# MAE comparison
comparison_df['MAE'].plot(kind='bar', ax=axes[1,0], color='lightgreen')
axes[1,0].set_title('MAE por Modelo')
axes[1,0].set_ylabel('MAE')
axes[1,0].tick_params(axis='x', rotation=45)

# Training Time comparison
if 'Training Time (s)' in comparison_df.columns:
    comparison_df['Training Time (s)'].plot(kind='bar', ax=axes[1,1], color='orange')
    axes[1,1].set_title('Tiempo de Entrenamiento')
    axes[1,1].set_ylabel('Segundos')
    axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Identificar mejor modelo

In [ ]:
# Identificar mejor modelo
best_model_r2 = comparison_df['R² Score'].idxmax()
best_model_rmse = comparison_df['RMSE'].idxmin()
best_model_mae = comparison_df['MAE'].idxmin()

print(f"\n🏆 MEJORES MODELOS POR MÉTRICA:")
print(f"   • Mejor R² Score: {best_model_r2} ({comparison_df.loc[best_model_r2, 'R² Score']:.4f})")
print(f"   • Mejor RMSE: {best_model_rmse} ({comparison_df.loc[best_model_rmse, 'RMSE']:.4f})")
print(f"   • Mejor MAE: {best_model_mae} ({comparison_df.loc[best_model_mae, 'MAE']:.4f})")

# Modelo recomendado (mejor balance)
comparison_df['Score_Compuesto'] = (
    comparison_df['R² Score'] * 0.4 +
    (1 - comparison_df['RMSE'] / comparison_df['RMSE'].max()) * 0.3 +
    (1 - comparison_df['MAE'] / comparison_df['MAE'].max()) * 0.3
)

best_overall = comparison_df['Score_Compuesto'].idxmax()
print(f"\n⭐ MODELO RECOMENDADO: {best_overall}")
print(f"   • Score Compuesto: {comparison_df.loc[best_overall, 'Score_Compuesto']:.4f}")


In [ ]:
# Análisis de residuos del mejor modelo
best_model_results = models_comparison[best_overall]
residuals = best_model_results['predictions'] - best_model_results['y_true']

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Scatter plot: Predicciones vs Valores reales
axes[0].scatter(best_model_results['y_true'], best_model_results['predictions'], 
               alpha=0.6, color='blue')
axes[0].plot([best_model_results['y_true'].min(), best_model_results['y_true'].max()], 
             [best_model_results['y_true'].min(), best_model_results['y_true'].max()], 
             'r--', lw=2)
axes[0].set_xlabel('Valores Reales')
axes[0].set_ylabel('Predicciones')
axes[0].set_title(f'Predicciones vs Reales - {best_overall}')

# Histograma de residuos
axes[1].hist(residuals, bins=30, alpha=0.7, color='green')
axes[1].set_xlabel('Residuos')
axes[1].set_ylabel('Frecuencia')
axes[1].set_title(f'Distribución de Residuos - {best_overall}')
axes[1].axvline(x=0, color='red', linestyle='--')

plt.tight_layout()
plt.show()

print(f"\n📊 ANÁLISIS DE RESIDUOS ({best_overall}):")
print(f"   • Media de residuos: {residuals.mean():.6f}")
print(f"   • Desviación estándar: {residuals.std():.4f}")
print(f"   • Residuos dentro de ±1σ: {(np.abs(residuals) <= residuals.std()).mean()*100:.1f}%")

# %%
# Análisis de importancia de features (si disponible)
if hasattr(regression_pipeline, 'get_feature_importance'):
    print("\n🔍 ANÁLISIS DE IMPORTANCIA DE FEATURES")
    print("=" * 50)
    
    try:
        feature_importance = regression_pipeline.get_feature_importance(best_overall)
        
        if feature_importance is not None:
            # Top 15 features más importantes
            top_features = feature_importance.head(15)
            
            plt.figure(figsize=(12, 8))
            top_features.plot(kind='barh', color='steelblue')
            plt.title(f'Top 15 Features Más Importantes - {best_overall}')
            plt.xlabel('Importancia')
            plt.tight_layout()
            plt.show()
            
            print("🔝 TOP 10 FEATURES MÁS IMPORTANTES:")
            for i, (feature, importance) in enumerate(top_features.head(10).items(), 1):
                print(f"   {i:2d}. {feature}: {importance:.4f}")
        
    except Exception as e:
        print(f"⚠️  No se pudo obtener importancia de features: {e}")


## Guardar resultados

In [ ]:
# Guardar resultados
results_summary = {
    'experiment_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'best_model': best_overall,
    'best_r2_score': comparison_df.loc[best_overall, 'R² Score'],
    'best_rmse': comparison_df.loc[best_overall, 'RMSE'],
    'best_mae': comparison_df.loc[best_overall, 'MAE'],
    'total_features': X_train.shape[1],
    'training_samples': X_train.shape[0],
    'test_samples': X_test.shape[0]
}

# Guardar comparación de modelos
comparison_df.to_csv('../data/processed/regression_models_comparison.csv')
print(f"📁 Comparación guardada en: ../data/processed/regression_models_comparison.csv")

# Guardar resumen
import json
with open('../data/processed/regression_results_summary.json', 'w') as f:
    json.dump(results_summary, f, indent=2)

print(f"📁 Resumen guardado en: ../data/processed/regression_results_summary.json")


In [ ]:
print("\n" + "="*60)
print("🎯 RESUMEN DE MODELOS DE REGRESIÓN")
print("="*60)
print(f"✅ Modelos entrenados: {len(models_comparison)}")
print(f"🏆 Mejor modelo: {best_overall}")
print(f"📊 Mejor R² Score: {comparison_df.loc[best_overall, 'R² Score']:.4f}")
print(f"📉 Mejor RMSE: {comparison_df.loc[best_overall, 'RMSE']:.4f}")
print(f"📈 Features utilizadas: {X_train.shape[1]}")
print(f"🔄 Listo para optimización en Fase 5")
print("="*60)

---

__Abraham Tartalos__